In [1]:
import os
os.chdir('../')
%pwd

'/media/Ubuntu/meg/mlprojects/Semantic_Segmentation_for_self_driving_cars'

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import glob

2023-09-25 13:28:03.829892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 13:28:04.473026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:

from src.components.data_preparation import DataIngestion

from src.components.callbacks import PrepareCallback
from src.components.model import PrepareModel
from src.config.configuration import ConfigurationManager
from src.entity.config_entity import DataIngestionConfig, PrepareCallbackConfig, PrepareModelConfig



In [4]:
from src.components.data_preparation import DataIngestion
from src.config.configuration import ConfigurationManager


In [5]:
class DataPreprocessing:
    def __init__(self, img_list: list, mask_list: list):
        self.img_list = img_list
        self.mask_list = mask_list

    @staticmethod
    def process_path(img_path, mask_path):
        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=3)
        img= tf.image.convert_image_dtype(img,tf.float32)

        mask = tf.io.read_file(mask_path)
        mask = tf.image.decode_png(mask, channels=3)
        mask = tf.math.reduce_max(mask,axis =-1, keepdims=True)

        return img, mask

    @staticmethod
    def preprocess(image, mask):
        input_image=tf.image.resize(image,(192,256), method='nearest')
        input_mask=tf.image.resize(mask,(192,256),method='nearest')

        return input_image, input_mask



    def data_preprocessing(self):

        #Combining images and masks into tensor pairs
        image_filenames = tf.constant(self.img_list)
        mask_filenames = tf.constant(self.mask_list)
        dataset = tf.data.Dataset.from_tensor_slices((image_filenames, mask_filenames))

            

        image_ds = dataset.map(self.process_path)
        processed_image_ds = image_ds.map(self.preprocess) 

        return processed_image_ds
    

         

In [6]:
config = ConfigurationManager()
prepare_data_config = config.get_data_ingestion_config()
prepare_dataset = DataIngestion(config=prepare_data_config)
imgs_list, masks_list = prepare_dataset.prepare_data()

[2023-09-25 13:28:05,904: INFO: common: yaml fileconfig.yamlloaded Successfully]
[2023-09-25 13:28:05,907: INFO: common: yaml fileparams.yamlloaded Successfully]
[2023-09-25 13:28:05,908: INFO: common: created directory at:artifacts]
[2023-09-25 13:28:05,909: INFO: common: created directory at:/media/Ubuntu/meg/mlprojects/Semantic_Segmentation_for_self_driving_cars/archive]


In [7]:
preprocessing = DataPreprocessing(imgs_list, masks_list)

In [8]:
dataset = preprocessing.data_preprocessing()

2023-09-25 13:28:07.978458: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-09-25 13:28:07.978509: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: megh
2023-09-25 13:28:07.978518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: megh
2023-09-25 13:28:07.978650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.86.10
2023-09-25 13:28:07.978674: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.86.10
2023-09-25 13:28:07.978681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.86.10


In [9]:
type(dataset)

tensorflow.python.data.ops.map_op._MapDataset